# RFC Simulation Notebook

This notebook lets you simulate any module (A–K) from Recursive Fractal Cosmology (RFC) using symbolic recursion kernels defined in your GitHub repo.

- Select a config JSON
- Load parameters
- Run the simulation
- Plot results like $\psi(t)$, $S(t)$, $m(\nu)$, or $\lambda_{div}(t)$

In [ ]:
# Imports
import json5
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from pathlib import Path

# Symbolic setup
t = sp.symbols('t')

In [ ]:
# Load all configs from single file and pick module I
config_path = Path('SimulationConfigs.json')  # now resides in the same notebooks/ folder
with open(config_path, 'r') as f:
    all_configs = json5.load(f)

config = next(cfg for cfg in all_configs if cfg["module"] == "I")
print(config)

In [ ]:
# Simulate ψ(t; ν) for mass field (Module I)

delta = config.get("delta", 4.669)
alpha = config.get("alpha", 0.01)
nu_values = config.get("nu_values", [0.003])
t_range = config.get("t_range", [0, 60])
dt = config.get("dt", 0.1)

t_vals = np.arange(t_range[0], t_range[1], dt)
results = {}

for nu in nu_values:
    psi_t = []
    for t_num in t_vals:
        val = sum((1/delta**j) * np.cos(j*t_num + nu) * np.exp(-alpha * j * t_num)
                  for j in range(1, 41))
        psi_t.append(val)
    results[nu] = psi_t

In [ ]:
# Plot
plt.figure(figsize=(10, 5))
for nu, psi_vals in results.items():
    plt.plot(t_vals, psi_vals, label=f"ν = {nu}")
plt.title("ψ(t; ν) — Symbolic Mass Field (Module I)")
plt.xlabel("t")
plt.ylabel("ψ(t)")
plt.legend()
plt.grid(True)
plt.show()